Cell 1: Install statsmodels

In [ ]:
!pip install statsmodels pandas matplotlib


Cell 2: Load and prepare data

In [ ]:
import pandas as pd

df = pd.read_csv('/content/merged_sentiment_price.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

# Use closing price only for ARIMA
df_arima = df[['date', 'close']].set_index('date')
df_arima = df_arima.asfreq('D').fillna(method='ffill')  # Fill missing days

df_arima.head()


Cell 3: Fit ARIMA model

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Simple ARIMA model (can tune p, d, q)
model = ARIMA(df_arima, order=(5, 1, 0))  # p=5, d=1, q=0
model_fit = model.fit()

print(model_fit.summary())


Cell 4: Forecast the next 30 days

In [ ]:
forecast = model_fit.forecast(steps=30)
forecast.index = pd.date_range(start=df_arima.index[-1] + pd.Timedelta(days=1), periods=30)

forecast_df = pd.DataFrame({
    "Forecast": forecast
})

forecast_df.head()


Cell 5: Plot results

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.plot(df_arima[-100:], label='Historical')
plt.plot(forecast_df, label='Forecast')
plt.title("ARIMA Forecast - 30 Days")
plt.legend()
plt.show()


Cell 6: Save model

In [ ]:
import joblib
joblib.dump(model_fit, "arima_model.pkl")


 Cell 7: Download

In [ ]:
from google.colab import files
files.download("arima_model.pkl")
